In [19]:
from datetime import datetime


class Account:
    """Класс для работы с банковским счётом"""
    
    def __init__(self, account_holder, balance=0):
        """
        Инициализация счёта
        
        account_holder: имя владельца
        balance: начальный баланс (по умолчанию 0)
        """
        if balance < 0:
            raise ValueError("Начальный баланс не может быть отрицательным")
        
        self.holder = account_holder
        self._balance = balance  # приватный атрибут для баланса
        self.operations_history = []  # история всех операций
        
        # Фиксируем начальный баланс в истории
        if balance > 0:
            self.operations_history.append({
                "type": "initial",
                "amount": balance,
                "timestamp": datetime.now(),
                "balance_after": self._balance,
                "status": "success",
                "used_credit": False
            })
    
    def deposit(self, amount):
        """Пополнение счёта"""
        # Проверка на положительную сумму
        if amount <= 0:
            self.operations_history.append({
                "type": "deposit",
                "amount": amount,
                "timestamp": datetime.now(),
                "balance_after": self._balance,
                "status": "fail",
                "used_credit": False
            })
            return False
        
        # Пополняем баланс
        self._balance += amount
        self.operations_history.append({
            "type": "deposit",
            "amount": amount,
            "timestamp": datetime.now(),
            "balance_after": self._balance,
            "status": "success",
            "used_credit": False
        })
        return True
    
    def withdraw(self, amount):
        """Снятие средств со счёта"""
        # Проверка на положительную сумму
        if amount <= 0:
            self.operations_history.append({
                "type": "withdraw",
                "amount": amount,
                "timestamp": datetime.now(),
                "balance_after": self._balance,
                "status": "fail",
                "used_credit": False
            })
            return False
        
        # Проверка наличия средств
        if amount > self._balance:
            self.operations_history.append({
                "type": "withdraw",
                "amount": amount,
                "timestamp": datetime.now(),
                "balance_after": self._balance,
                "status": "fail",
                "used_credit": False
            })
            return False
        
        # Снимаем средства
        self._balance -= amount
        self.operations_history.append({
            "type": "withdraw",
            "amount": amount,
            "timestamp": datetime.now(),
            "balance_after": self._balance,
            "status": "success",
            "used_credit": False
        })
        return True
    
    def get_balance(self):
        """Возвращает текущий баланс"""
        return self._balance
    
    def get_history(self):
        """Возвращает копию истории операций"""
        return self.operations_history.copy()


# Создание обычного счёта
name = input("Имя владельца счёта: ")
balance = float(input("Начальный баланс: "))

account = Account(name, balance)
print("Создан счёт:", account.holder, "| Баланс:", account.get_balance())


Создан счёт: ррррр | Баланс: 999.0


In [20]:
# Проверка работы методов класса Account
print("Текущий счёт:", account.holder)
print("Баланс:", account.get_balance())

# Тест метода deposit
dep_amount = float(input("Сумма пополнения: "))
result = account.deposit(dep_amount)
print("Результат deposit:", result)
print("Новый баланс:", account.get_balance())

# Тест метода withdraw
with_amount = float(input("Сумма снятия: "))
result = account.withdraw(with_amount)
print("Результат withdraw:", result)
print("Новый баланс:", account.get_balance())

# Вывод истории операций
print("\nИстория операций:")
for op in account.get_history():
    print(f"{op['type']}: {op['amount']}, статус: {op['status']}, "
          f"баланс после: {op['balance_after']}, "
          f"время: {op['timestamp'].strftime('%d.%m.%Y %H:%M:%S')}")


Текущий счёт: ррррр
Баланс: 999.0
Результат deposit: True
Новый баланс: 1998.0
Результат withdraw: True
Новый баланс: 1098.0

История операций:
initial: 999.0, статус: success, баланс после: 999.0, время: 27.12.2025 21:50:49
deposit: 999.0, статус: success, баланс после: 1998.0, время: 27.12.2025 21:51:09
withdraw: 900.0, статус: success, баланс после: 1098.0, время: 27.12.2025 21:51:11


In [21]:
class CreditAccount(Account):
    """Класс кредитного счёта, наследует Account"""
    
    def __init__(self, account_holder, balance=0, credit_limit=0):
        """
        Инициализация кредитного счёта
        
        account_holder: имя владельца
        balance: начальный баланс
        credit_limit: кредитный лимит
        """
        if balance < 0:
            raise ValueError("Начальный баланс не может быть отрицательным")
        if credit_limit < 0:
            raise ValueError("Кредитный лимит не может быть отрицательным")
        
        super().__init__(account_holder, balance)
        self.credit_limit = credit_limit
    
    def withdraw(self, amount):
        """Снятие средств с учётом кредитного лимита"""
        # Проверка на положительную сумму
        if amount <= 0:
            self.operations_history.append({
                "type": "withdraw",
                "amount": amount,
                "timestamp": datetime.now(),
                "balance_after": self._balance,
                "status": "fail",
                "used_credit": False
            })
            return False
        
        # Проверка кредитного лимита
        if amount > self._balance + self.credit_limit:
            self.operations_history.append({
                "type": "withdraw",
                "amount": amount,
                "timestamp": datetime.now(),
                "balance_after": self._balance,
                "status": "fail",
                "used_credit": False
            })
            return False
        
        # Определяем, использовался ли кредит
        used_credit = self._balance < amount
        
        # Снимаем средства
        self._balance -= amount
        self.operations_history.append({
            "type": "withdraw",
            "amount": amount,
            "timestamp": datetime.now(),
            "balance_after": self._balance,
            "status": "success",
            "used_credit": used_credit
        })
        return True
    
    def get_available_credit(self):
        """Возвращает доступный кредит (баланс + лимит)"""
        return self._balance + self.credit_limit


# Создание кредитного счёта
credit_name = input("Имя владельца кредитного счёта: ")
credit_balance = float(input("Начальный баланс кредитного счёта: "))
credit_limit = float(input("Кредитный лимит: "))

credit_acc = CreditAccount(credit_name, credit_balance, credit_limit)
print("Создан кредитный счёт:", credit_acc.holder)
print("Баланс:", credit_acc.get_balance())
print("Доступно:", credit_acc.get_available_credit())


Создан кредитный счёт: оооо
Баланс: 99999.0
Доступно: 199998.0


In [22]:
# Проверка работы кредитного счёта
print("Кредитный счёт:", credit_acc.holder)
print("Баланс:", credit_acc.get_balance())
print("Доступно:", credit_acc.get_available_credit())

# Снятие в пределах баланса
amount1 = float(input("Снять в пределах баланса: "))
result1 = credit_acc.withdraw(amount1)
print("Результат:", result1)
print("Баланс:", credit_acc.get_balance())

# Снятие с использованием кредита
amount2 = float(input("Снять с использованием кредита: "))
result2 = credit_acc.withdraw(amount2)
print("Результат:", result2)
print("Баланс:", credit_acc.get_balance())
print("Доступно:", credit_acc.get_available_credit())

# Вывод истории с отметкой об использовании кредита
print("\nИстория кредитного счёта:")
for op in credit_acc.get_history():
    credit_info = " (использован кредит)" if op["used_credit"] else ""
    print(f"{op['type']}: {op['amount']}, статус: {op['status']}, "
          f"баланс: {op['balance_after']}{credit_info}")


Кредитный счёт: оооо
Баланс: 99999.0
Доступно: 199998.0
Результат: True
Баланс: 99900.0
Результат: True
Баланс: 98901.0
Доступно: 198900.0

История кредитного счёта:
initial: 99999.0, статус: success, баланс: 99999.0
withdraw: 99.0, статус: success, баланс: 99900.0
withdraw: 999.0, статус: success, баланс: 98901.0
